# 0. Import/Classes/Logger

In [10]:
from bs4 import BeautifulSoup as BS
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import (TimeoutException,
                                        ElementNotVisibleException, 
                                        ElementNotSelectableException, 
                                        NoSuchElementException, 
                                        ElementNotInteractableException, 
                                        StaleElementReferenceException, 
                                        ElementClickInterceptedException, 
                                        InvalidSelectorException, 
                                        WebDriverException,
                                        NoSuchWindowException
                                        )
from urllib3.exceptions import NewConnectionError, MaxRetryError
from typing import NamedTuple
from zipfile import ZipFile, ZIP_DEFLATED
import os, signal, sys, psutil
import re
from time import sleep as time_sleep, time
from datetime import datetime
from collections import deque
import pickle
from tqdm import tqdm
from itertools import chain
import pandas as pd
import numpy as np
import logging
from pathlib import Path

In [11]:
# получение пользовательского логгера и установка уровня логирования
short_logger = logging.getLogger('short_logger')
short_logger.setLevel(logging.INFO)

# настройка обработчика и форматировщика в соответствии с нашими нуждами
short_handler = logging.FileHandler("data/short_logger.log", encoding='utf-8', mode='a')
short_formatter = logging.Formatter("%(asctime)s %(levelname)s %(funcName)s %(message)s")

# добавление форматировщика к обработчику 
short_handler.setFormatter(short_formatter)
# добавление обработчика к логгеру
short_logger.addHandler(short_handler)

# получение пользовательского логгера и установка уровня логирования
long_logger = logging.getLogger('long_logger')
long_logger.setLevel(logging.WARNING)

# настройка обработчика и форматировщика в соответствии с нашими нуждами
long_handler = logging.FileHandler("data/long_logger.log", encoding='utf-8', mode='w')
long_formatter = logging.Formatter("%(asctime)s %(levelname)s %(funcName)s %(message)s")

# добавление форматировщика к обработчику 
long_handler.setFormatter(long_formatter)
# добавление обработчика к логгеру
long_logger.addHandler(long_handler)

In [12]:
def _save_df_to_zip(df_: pd.DataFrame, archive_name: str = 'archive', folder: str='data', replace: bool=False) -> None:
    # Путь к файлу
    file_path = Path(folder).joinpath(archive_name + '.zip')
    Path(folder).mkdir(exist_ok=True)
    # Проверяем, существует ли файл
    if file_path.exists() and not replace:
        # Получаем время создания файла
        time = datetime.fromtimestamp(file_path.lstat().st_atime).strftime('%Y-%m-%d %H:%M')

        # Создаем новое имя файла с добавлением времени Unix
        new_file_name = file_path.stem + "_" + str(time) + file_path.suffix

        # Создаем новый путь для переименованного файла
        new_file_path = file_path.with_name(new_file_name)
        # Переименовываем файл
        file_path.rename(new_file_path)

# to csv
    compression_opts = dict(method='zip', archive_name=f'{archive_name}.csv')
    df_.to_csv(f'{folder}/{archive_name}.zip', index=False, compression=compression_opts, encoding='utf-8')
    

def occupied_memory() -> float:
    # Получить список всех процессов
    all_processes = psutil.process_iter()

    # Пройтись по каждому процессу и найти процессы с именем "chrome.exe"
    chrome_processes = [p for p in all_processes if p.name() == "chrome.exe"]

    # Инициализировать переменную для хранения общего объема памяти
    total_memory = 0

    # Получить информацию о памяти для каждого процесса Chrome

    if chrome_processes:
        for process in chrome_processes:
            try:
                memory_info = process.memory_info()
            except psutil.NoSuchProcess:
                memory_usage = 0
            else:
                memory_usage = memory_info.rss
                total_memory += memory_usage

    # Преобразовать размер в удобочитаемый формат
    # formatted_size = psutil._common.bytes2human(total_memory)
    return (total_memory) / (1024**2)

def kill_chrome():
    # Получить список всех процессов
    all_processes = psutil.process_iter()

    # Пройтись по каждому процессу и найти процессы с именем "chrome.exe"
    chrome_processes = [p for p in all_processes if p.name() == "chrome.exe"]

    # Получить информацию о памяти для каждого процесса Chrome

    if chrome_processes:
        for process in chrome_processes:
            try:
                pid = process.pid
            except psutil.NoSuchProcess:
                pass
            else:
                os.kill(pid, signal.SIGINT)
                


def reload_driver(driver: webdriver.Chrome, time_wait: int=5) -> webdriver.Chrome:
    if occupied_memory() > 0:
        kill_chrome()
    driver = webdriver.Chrome()
    driver.maximize_window()
    while occupied_memory() <= 1:
        time_sleep(0.5)
    time_sleep(time_wait)
    return driver

def _check_load_page(driver: webdriver.Chrome, timeout: int=20, limitMB: int=1400) -> bool:
    """
       Проверка наличия названия продукта в ячейке
    """
    time_start = int(time())
    while (int(time()) - time_start) < timeout:
        n = driver.find_elements(By.XPATH, """//div[contains(@class, "p-dsk-srch-retailer__content")]
                                                //div[contains(@class, "b-dsk-grid__container")]
                                                /a[contains(@class, "p-dsk-srch-retailer__card")]
                                                //div[contains(@class, "b-srch-card__price-new")]
                                                //span[@data-test-ref="money-base"]
                                                """)
        
        # Проверка на переполнение памяти:
        if occupied_memory() > limitMB:
            raise WebDriverMemoryOut('Превышение лимита')
            
        if len(n) > 0:
            if int(n[0].text.replace('\xa0', '').replace(' ', '')) > 0:
                return True
        else:
            time_sleep(0.2)
            continue
    return False

In [13]:
class market_link(NamedTuple):
    sity: str
    title: str
    discounts: int
    alcohol: bool
    href: str
    su1: str
    su2: str
    su3: str
    page_href: str

class MyTimeoutError(Exception):
    def __init__(self, text: str='timeout'):
        self.txt = text

class MyLinkDifferError(Exception):
    def __init__(self, text: str='Link Differ'):
        self.txt = text

class WebDriverMemoryOut(Exception):
    def __init__(self, text: str='Web Driver Memory Out'):
        self.txt = text

In [6]:
driver = webdriver.Chrome()
driver.maximize_window()

# 1. Получение адресов городов

# 2. Получение ссылок на Магазины

## 2.1. Получение ссылки на старицы каталогов с магазинами

### 2.1.1. Функции

In [86]:
def _get_page_count(driver: webdriver.Chrome, url: str, pagin_sufix: str, timeout: int=30) -> list[str]:
    """
        Возвращает лист с доступными ссылками
    """
    # Класс задержки, который будет ожидать когда догрузится элемент
    wait = WebDriverWait(
        driver,
        timeout=timeout + 10,
        poll_frequency=1,
        # ignored_exceptions=[ElementNotVisibleException, ElementNotSelectableException, NoSuchElementException]
        )
    # Вызов страницы
    driver.get(url)
    time_start = int(time())
    while (int(time()) - time_start) < timeout:
        # Задержка действий до загрузки отслеживаемых елементов.
        # wait.until(lambda d: d.find_element(by=By.XPATH, value="//nav[@aria-label='Постраничная навигация']"))
        wait.until(lambda d: d.find_element(by=By.XPATH, value="//nav[@data-test-ref='paginator']"))
        # Парсинг
        page = BS(driver.page_source, 'html.parser')
        
        try:
            f1 = page.find('nav', {'data-test-ref':"paginator"})
        except Exception as ex:
            print(ex)
            continue
            
        try:    
            pagination = int(f1.find_all('a')[-2].text)
        except Exception as ex:
            print(ex)
            continue
        
        out_list = [f'{url}{pagin_sufix}{x}' for x in range(1, pagination+1)]
        
        break
    else:
        raise Exception("timeout")
    
    return out_list

### 2.1.2. Выполнение

In [87]:
# driver.get('https://edadeal.ru/lipeck/retailers')
url = 'https://edadeal.ru/lipeck/retailers'

In [88]:
temp_url_list = _get_page_count(driver, url, '?page=')
temp_url_list

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range


['https://edadeal.ru/lipeck/retailers?page=1',
 'https://edadeal.ru/lipeck/retailers?page=2',
 'https://edadeal.ru/lipeck/retailers?page=3',
 'https://edadeal.ru/lipeck/retailers?page=4',
 'https://edadeal.ru/lipeck/retailers?page=5']

## 2.2. Получение непосредственно все ссылки на Магазины

### 2.2.1. Функции

In [89]:
def get_retailers_links(driver: webdriver.Chrome, url: str, sity: str, timeout: int=30) -> list[market_link]:
    """
        Возвращает лист с магазинами и ссылками.
    """
    
    host = '/'.join(url.split('/')[:-1])
    out_list = list()
    # Класс задержки, который будет ожидать когда догрузится элемент
    wait = WebDriverWait(
        driver,
        timeout=timeout + 10,
        poll_frequency=1
        )

    
    # Вызов страницы
    driver.get(url)
    time_start = int(time())
    
    while (int(time()) - time_start) < timeout:
        # Задержка действий до загрузки отслеживаемых елементов.
        wait.until(lambda d: d.find_element(by=By.XPATH, value="//nav[@aria-label='Постраничная навигация']"))
        # Парсинг
        page = BS(driver.page_source, 'html.parser')
        
        try:
            f1 = page.find('div', class_='p-dsk-srch-retailers__content').find('div', class_='b-dsk-grid__container').find_all('article')
        except Exception as ex:
            print(ex)
            continue
        
        try:    
            for item in f1:
                href = host + item.find('a').get('href')
                title = item.find('a').find('h4').get('title')
                discounts = int((re.findall(r'(\d+)', item.find('a').find('p').text)[0]))
                out_list.append(market_link(sity, title, discounts, False, href, None, None, None, None))
                
        except Exception as ex:
            print(ex)
            continue
        
        break
    
    else:
        raise MyTimeoutError('timeout')
    
    return out_list

### 2.2.2. Выполнение

In [90]:
temp_retailer_links = list()
for link in temp_url_list:
    temp_retailer_links.append(get_retailers_links(driver, link, 'lipetsk'))

retailer_links = list(chain.from_iterable(temp_retailer_links))
print(len(retailer_links))
# print(retailer_links)

'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
59


### 2.2.3. Save/Load

In [91]:
with open('data/retailer_links.pickle', 'wb') as file:
    pickle.dump(retailer_links, file)

In [92]:
with open('data/retailer_links.pickle', 'rb') as file:
    retailer_links = pickle.load(file)

In [13]:
retailer_links[1]

market_link(sity='lipetsk', title='Магнит Косметик', discounts=3528, alcohol=False, href='https://edadeal.ru/lipeck/retailers/magnit-cosmo', su1=None, su2=None, su3=None, page_href=None)

# 3. Получение адресов каталогов товаров

## 3.1. Функции

In [93]:
def get_link_to_pages_without_except(driver: webdriver.Chrome, market: market_link, effort: int=10, timeout: int=20, limitMB: int=1400):
    """
         Функция получает ссылки на каталоги товаров в каждом магазине,
         В детализации до SU1/SU2/SU3, если таковые присутствуют.

         Для того что бы обойти ошибки, которые возникают при сборе информации,
         присутствует подфункция, которая перезапускается и начинает сбор данных с последней
         "точки сохранения".
    """
                        
    out_list = list()                # Лист в который собираются данные из подфункции
    HEAD = None                      # Точка сохранения, она опирается на группу SU1
    reload_driver_bool = False       # Флаг обозначающий, что был перезагружен драйвер selenium
    
    def _get_link_to_pages(driver: webdriver.Chrome, market: market_link, timeout: int=20, limitMB: int=1400):
        """
            ПодФункция которая непосредственно исследует страницу с магазином,
            Проходит по категориям SU1, SU2 и достает ссылки на категории SU3, если есть.

            В процессе возникает множество ошибок,
            Которые отлавливаются либо внутри, этой функции, либо во внешней функции.
            
        """

        # Модуль, который позволяет дождаться когда прогрузится необъодимый элемент
        wait = WebDriverWait(
            driver,
            timeout=timeout,
            poll_frequency=1
            )

        alcohol = False       # Рудимент, необъодимо убрать

        # Проброска переменных из внешней функции
        nonlocal out_list
        nonlocal HEAD
        nonlocal reload_driver_bool
        
        # Находим все SU1 категории, исключая первую: "ВСЕ"
        el1 = (driver
              .find_element(By.CSS_SELECTOR, '.p-dsk-srch-retailer__body')
              .find_element(By.CSS_SELECTOR, '.p-dsk-srch-retailer__sidebar-container')
              .find_element(By.CSS_SELECTOR, '.i-block-helper')
              .find_element(By.CSS_SELECTOR, '.b-dsk-srch-cats-tree__node')
              .find_elements(By.XPATH, "following-sibling::*")
              )
        
        # Активируем режим +18
        if not alcohol:
            for _ in range(20):
                try:
                    driver.find_element(By.XPATH, '//div[contains(text(), "Да. Мне есть 18")]/ancestor::button').click()
                except NoSuchElementException:
                    time_sleep(0.2)
                    continue
                except ElementNotInteractableException:
                    time_sleep(0.2)
                    continue
                else:
                    alcohol = True
                    break
        
        # Цикл по каждой SU1 категории отдельно
        for el in el1:
            
            # Временный лист, в который будет собираться информация о SU2 и SU3
            temp_list = list()
            
            # Кликаем на категории, что бы открыть список подкатегорий SU2
            el2 = (el
                    .find_element(By.XPATH, './/div[contains(@class, "b-dsk-srch-cats-tree__item-wrapper")]/button')
                    )
            su1 = el2.text
            
            # Проверка перезагрузки драйвера
            # И продолжение с места остановки (SU1)
            if not HEAD:
                reload_driver_bool = False
            
            if reload_driver_bool:
                if not (HEAD == su1):
                    continue
                else:
                    reload_driver_bool = False

            # Продолжение с места остановки (SU1)
            el2.click()
            time_sleep(0.2)

            # Проверка, что загрузились товары, а так же проверка на превышения лимита объема оперативной памяти.
            check_page = _check_load_page(driver, timeout, limitMB)
            
            if check_page:
                
                # Фиксация "точки сохранения", возможно стоило бы перенести точку сохранения выше, это нужно проверить.
                HEAD = su1

                # Находим ссылки на SU2, (в будующем необходимо переписать название переменных)
                try:
                    el3 = (el
                        .find_element(By.XPATH, './/div[contains(@class, "b-dsk-srch-cats-tree__children")]/div[contains(@class, "i-block-helper")]')
                        .find_elements(By.XPATH, './/div[contains(@class, "b-dsk-srch-cats-tree__node")]//button')
                        )
                except NoSuchElementException:
                    # Редко, но бывает, что подкатегорий нет.
                    short_logger.info(f'only su1: ::  market: {market.title} ::  HEAD: {HEAD} ::  href: {market.href}')
                    href = driver.current_url
                    su2 = np.nan
                    su3 = np.nan
                    temp_list.append(market_link(
                                                    market.sity,
                                                    market.title,
                                                    market.discounts,
                                                    market.alcohol,
                                                    market.href,
                                                    su1,
                                                    su2,
                                                    su3,
                                                    href
                                                ))
                    continue
                    
            else:
                raise MyTimeoutError()         # Информация на страницу так и не загрузиласть

            # SU2
            for item in el3:
                # Проходим по каждой  SU2
                for _ in range(10):
                    try:
                        item.click()
                    except ElementClickInterceptedException:
                        # Иногда категория не открывается, потому что она не видна на экране
                        body = driver.find_element(By.XPATH, '//body')
                        body.send_keys(Keys.ARROW_DOWN)
                        time_sleep(0.2)
                        continue
                    else:
                        break
                else:
                    # Иногда, что-то еще.
                    raise ElementClickInterceptedException()

                # С после нажатия на категорию, мы прогручиваем минисписок вниз на один элемент, что бы показалась следующая SU2
                time_sleep(0.2)
                item.send_keys(Keys.ARROW_DOWN)

                # Проверяем загрузку товаров при выборе категории SU2
                check_page = _check_load_page(driver, timeout, limitMB)
                                
                if check_page:
                    # Достаем SU2
                    su2 = item.text

                    # Достаем список с категориями SU3,
                    su3_list = wait.until(lambda d: d.find_elements(By.XPATH, '//li[contains(@class, "b-dsk-srch-cats-list__item")]/button'))
    
                    # Если 3 и более елементов в списке, это означает, что у категории SU2 есть подкатегории SU3.
                    if len(su3_list) > 2:
                        for s_item in su3_list[1:]:
                            su3 = s_item.text
                            # Таким способом мы добывает ссылки на каталог с детализацией до SU3
                            # Потом еще предстоит пройтись по каждой странице каталога, но если добавить эту операцию сюда,
                            # это сильно уложнит выполнения кода, и точки сохранения, так как малое количество памяти накладывает свои ограничения.
                            href = driver.current_url + '&segmentUuid2='+ s_item.get_attribute('value')
                            temp_list.append(market_link(
                                                            market.sity,
                                                            market.title,
                                                            market.discounts,
                                                            market.alcohol,
                                                            market.href,
                                                            su1,
                                                            su2,
                                                            su3,
                                                            href
                                                        ))
                    else:
                        su3 = np.nan
                        href = driver.current_url
                        temp_list.append(market_link(
                                                        market.sity,
                                                        market.title,
                                                        market.discounts,
                                                        market.alcohol,
                                                        market.href,
                                                        su1,
                                                        su2,
                                                        su3,
                                                        href
                                                    ))
            
            # Добавляем информацию во внешний лист
            out_list.append(temp_list)

        # Оповещаем внешнюю функцию, что в этом магазине закончились категории SU1
        HEAD = 'finally'
        # ====================================================================== Конец

    # Цикл который будет пытаться занова начать собирать информацию, пока не истечет количество попыток
    # Что делать когда исзасходуется effort???????
    for _ in range(effort):
        # Закончились ли категории SU1
        if not (HEAD == 'finally'):
            # Перезагрузка драйвера
            driver = reload_driver(driver)
            try:
                # Открытие ссылки с магазином
                driver.get(market.href)
            except WebDriverException as ex:
                short_logger.error(f'WebDriverException: {ex} ::  market: {market.title} ::  HEAD: {HEAD} ::  href: {market.href}')
                long_logger.error(f'WebDriverException: {ex} ::  market: {market.title} ::  HEAD: {HEAD} ::  href: {market.href}', exc_info=True)
                continue

            # Первая загрузка магазина особенно долгая, по этому закладываем дополнительное время на проверку.
            if not _check_load_page(driver, timeout=120):
                short_logger.error(f'WebDriver TimeOut: ::  market: {market.title} ::  HEAD: {HEAD} ::  href: {market.href}')
                long_logger.error(f'WebDriver TimeOut: ::  market: {market.title} ::  HEAD: {HEAD} ::  href: {market.href}', exc_info=True)
                continue
            
            try:
                driver.find_element(By.XPATH, '//p[contains(text(), "В этом магазине сейчас нет акций")]')
            except NoSuchElementException:
                pass
            else:
                short_logger.warning(f'В этом магазине сейчас нет акций: ::  market: {market.title} ::  HEAD: {HEAD} ::  href: {market.href}')
                continue
            
            reload_driver_bool = True
            try:
                _get_link_to_pages(driver, market, timeout, limitMB)
            # except WebDriverException as ex:
            #     print('_get_link_to_pages WebDriverException', ex)
            #     continue
            except WebDriverMemoryOut as ex:
                short_logger.info(f'WebDriverMemoryOut: {ex} ::  market: {market.title} ::  HEAD: {HEAD} ::  href: {market.href}')
                continue
            except StaleElementReferenceException as ex:
                short_logger.warning(f'StaleElementReferenceException: {ex} ::  market: {market.title} ::  HEAD: {HEAD} ::  href: {market.href}')
                long_logger.warning(f'StaleElementReferenceException: {ex} ::  market: {market.title} ::  HEAD: {HEAD} ::  href: {market.href}', exc_info=True)
                continue
            except ElementClickInterceptedException as ex:
                short_logger.error(f'ElementClickInterceptedException: {ex} ::  market: {market.title} ::  HEAD: {HEAD} ::  href: {market.href}')
                long_logger.error(f'ElementClickInterceptedException: {ex} ::  market: {market.title} ::  HEAD: {HEAD} ::  href: {market.href}', exc_info=True)
                continue
            else:
                driver.quit()
        else:
            return out_list
                     

## 3.2. Выполнение

In [94]:
dead_line = 20
out_list = list()

market_deque = deque(retailer_links)

with tqdm(total=len(market_deque)) as pbar:
    while len(market_deque) > 0:
        if not dead_line:
            break
        
        market = market_deque.popleft()
        
        if market.discounts > 0:
            # print("Маркет: ", market.title)
            out = get_link_to_pages_without_except(driver, market)
            
            if out is None:
                market_deque.append(market)
                dead_line -= 1
                continue
            
            out_list.extend(out)
        pbar.update(1)

100%|███████████████████████████████████████████████████████████████████████████████| 59/59 [1:48:38<00:00, 110.48s/it]


In [106]:
market_catalog_links = list(chain.from_iterable(out_list.copy()))
print(f'Всего ссылок: {len(market_catalog_links)}')
for item in market_catalog_links:
    if not isinstance(item, market_link):
        print(item)


Всего ссылок: 3653
market_link(sity='lipetsk', title='Перекрёсток', discounts=1682, alcohol=False, href='https://edadeal.ru/lipeck/retailers/perekrestok', su1='Продукты', su2='Молочные продукты', su3='Сыр', page_href='https://edadeal.ru/lipeck/retailers/perekrestok?segmentUuid=3b31b1a8-6311-11e6-849f-52540010b608&segmentUuid2=3b31b30e-6311-11e6-849f-52540010b608')
market_link(sity='lipetsk', title='Перекрёсток', discounts=1682, alcohol=False, href='https://edadeal.ru/lipeck/retailers/perekrestok', su1='Продукты', su2='Молочные продукты', su3='Йогурт', page_href='https://edadeal.ru/lipeck/retailers/perekrestok?segmentUuid=3b31b1a8-6311-11e6-849f-52540010b608&segmentUuid2=3b31b862-6311-11e6-849f-52540010b608')
market_link(sity='lipetsk', title='Перекрёсток', discounts=1682, alcohol=False, href='https://edadeal.ru/lipeck/retailers/perekrestok', su1='Продукты', su2='Молочные продукты', su3='Творог', page_href='https://edadeal.ru/lipeck/retailers/perekrestok?segmentUuid=3b31b1a8-6311-11e6-8

## 3.1. Save/Load

In [96]:
with open('data/market_catalog_links.pickle', 'wb') as file:
    pickle.dump(market_catalog_links, file)

In [14]:
with open('data/market_catalog_links.pickle', 'rb') as file:
    market_catalog_links = pickle.load(file)

In [15]:
market_catalog_links[400]

market_link(sity='lipetsk', title='Магнит', discounts=32029, alcohol=False, href='https://edadeal.ru/lipeck/retailers/magnit-univer', su1='Продукты', su2='Напитки', su3='Холодный чай', page_href='https://edadeal.ru/lipeck/retailers/magnit-univer?segmentUuid=3b336f02-6311-11e6-849f-52540010b608&segmentUuid2=3b3375a8-6311-11e6-849f-52540010b608')

# 4. Получение информации о товарах и ссылки на них

## 4.1 Функции

In [16]:
def parser_sku_01(page_source: str, market: market_link, date: datetime) -> pd.DataFrame:
    """
        Извлечение данных из страницы,
        Данные возвращаются в виде pd.DataFrame
    """
    
    # Получаем хост из ссылки
    host = '/'.join(market.page_href.split('/', maxsplit=3)[:3])
    # 
    page = BS(page_source, 'html.parser')
    
    # Находим карточки с продуктами
    f1 = page.find('div', {'data-test-ref':'grid'})
    f2 = f1.find('div').find_all('a', {'data-test-ref':'b-srch-card'})
    
    out_list = list()
    
    for f3 in f2:
        # название
        title = f3.find('div', class_='b-srch-card__title').get_text(strip=True)
        
        # дата до истечения
        try:
            date_to = datetime.strptime(f3.find('time').get('datetime'), '%Y-%m-%d')
        except AttributeError:
            date_to = np.nan
        
        # Цена (разбита на части)
        try:
            b = (f3
                .find('div', class_="b-srch-card__price")
                .find('div', class_="b-srch-card__price-new")
                .find('div', class_='b-money__root-wrapper')
                .find('span', {'data-test-ref':"money-base"})
                .get_text(strip=True)
                .replace('\xa0', '')
                )
        except AttributeError as ex:
            price = np.nan
            short_logger.critical(f"no price, {ex}: {market.title} :: {market.page_href}")
            long_logger.critical(f"no price, {ex}: {market.title} :: {market.page_href}", exc_info=True)
        
        else:
            try:
                s = (f3
                    .find('div', class_="b-srch-card__price")
                    .find('div', class_="b-srch-card__price-new")
                    .find('div', class_='b-money__root-wrapper')
                    .find('span', {'data-test-ref':"money-subunit"})
                    .get_text(strip=True)
                    .replace('\xa0', ''))
            except AttributeError:
                s = '0'
                
            price = float(f'{b}.{s}')
        
        # -------------------------------------------------------------------------------------------------
        # Предыдущая цена (разбита на части)
        try:
            b_last = (f3
                      .find('div', class_="b-srch-card__price")
                      .find('div', class_="b-srch-card__price-old")
                      .find('div', class_='b-money__root-wrapper')
                      .find('span', {'data-test-ref':"money-base"})
                      .get_text(strip=True)
                      .replace('\xa0', '')
                      )
        except AttributeError as ex:
            price_last = np.nan
        
        else:
            try:
                s_last = (f3
                          .find('div', class_="b-srch-card__price")
                          .find('div', class_="b-srch-card__price-old")
                          .find('div', class_='b-money__root-wrapper')
                          .find('span', {'data-test-ref':"money-subunit"})
                          .get_text(strip=True)
                          .replace('\xa0', '')
                          )
            except AttributeError:
                s_last = '0'
                
            price_last = float(f'{b_last}.{s_last}')

        # -------------------------------------------------------------------------------------------------
        # тип валюты
        try:
            unit = f3.find('div', class_='b-money__root-wrapper').find('span', {'data-test-ref':"currency"}).get_text(strip=True)
        except AttributeError:
            unit = np.nan

        # -------------------------------------------------------------------------------------------------
        # скидка
        try:
            sale = f3.find('div', class_='b-srch-card__discount-label').get_text(strip=True)
        except AttributeError:
            sale = np.nan
        
        # -------------------------------------------------------------------------------------------------
        # Условие
        try:
            condition = (f3
                         .find('div', class_="b-srch-card__content-container")
                         .find('div', class_="b-tpl-offer-label__text")
                         .get_text(strip=True)
                         .replace('\xa0', '')
                         )
        except AttributeError as ex:
                condition = np.nan

        # -------------------------------------------------------------------------------------------------
        # вес
        try:
            f4 = (f3
                .find('div', class_='b-srch-card__info')
                .find('span')
                .get_text(strip=True)
                .replace('\xa0', ' ')
                .split('•')
                )
            vol = f4[0].strip()
        
        # руб/кг. руб/шт.
            vol_vs = f4[1].strip()
        except AttributeError:
            vol = np.nan
            vol_vs = np.nan
                
        # -------------------------------------------------------------------------------------------------
        # ссылка
        try:
            href = host + f3.get('href')
        except AttributeError as ex:
            href = np.nan
        
                
        out_list.append((date, market.sity, market.title, market.su1, market.su2, market.su3, title, date_to, condition ,price, price_last, unit, sale, vol, vol_vs, href))


    return pd.DataFrame(out_list, columns=['date', 'sity', 'market', 'SU1', 'SU2', 'SU3', 'title', 'date_to', 'condition', 'price', 'price_last', 'unit', 'sale', 'vol', 'vol_vs', 'href'])






def get_data(driver: webdriver.Chrome, market: market_link, effort: int=10, timeout: int=20, limitMB: int=1400):
    need_reload_driver = False
    need_return_driver = False
    df_list = list()
    
    def get_data_worker(driver: webdriver.Chrome, market: market_link, timeout: int=20, limitMB: int=1400):
        driver.get(market.page_href)
        
        def check_alcohol(driver: webdriver.Chrome, market: market_link, timeout: int=20, limitMB: int=1400):
            # Активируем режим +18
            if market.su1 == 'Алкоголь':
                for _ in range(40):
                    try:
                        button = driver.find_element(By.XPATH, '//div[contains(text(), "Да. Мне есть 18")]/ancestor::button')
                        button.click()
                    except NoSuchElementException:
                        time_sleep(0.5)
                        continue
                    except ElementNotInteractableException:
                        time_sleep(0.5)
                        continue
                    else:
                        if button.get_attribute('aria-expanded') == 'true':
                            if not _check_load_page(driver, timeout, limitMB):
                                raise MyTimeoutError()
                            break
                        else:
                            continue
                else:
                    raise MyTimeoutError()
            else:
                if not _check_load_page(driver, timeout, limitMB):
                    raise MyTimeoutError()
                
        check_alcohol(driver, market, timeout, limitMB)
        
        if not (driver.current_url == market.page_href):
            raise MyLinkDifferError()
        
        find = driver.find_elements(By.XPATH, """//nav[contains(@class, "b-dsk-paginator")]
                                                 /a[contains(@class, "b-dsk-paginator__item")]
                                                        """)
        df_list.append(parser_sku_01(driver.page_source, market, datetime.now()))
        if len(find):
            for n in range(2, len(find)):
                link = f'?page={n}&'.join(market.page_href.split('?'))
                driver.get(link)
                check_alcohol(driver, market, timeout, limitMB)
                df_list.append(parser_sku_01(driver.page_source, market, datetime.now()))
                
        return df_list
    # END ===================================================================================== get_data_worker


    for _ in range(effort):
        if need_reload_driver:
            need_reload_driver = False
            need_return_driver = True
            driver = reload_driver(driver)
        
        try:
            out_list = get_data_worker(driver, market, timeout, limitMB)
        
        except MyTimeoutError:
            exc_type, exc_obj, exc_tb = sys.exc_info()
            short_logger.info(f'MyTimeoutError: {exc_type.__name__} ::  market: {market.title} ::  href: {market.href}')
            long_logger.info(f'MyTimeoutError: {exc_type.__name__} ::  market: {market.title} ::  href: {market.href}', exc_info=True)
            continue
            
        except MaxRetryError:
            need_reload_driver = True
            exc_type, exc_obj, exc_tb = sys.exc_info()
            short_logger.error(f'MaxRetryError: {exc_type.__name__} ::  market: {market.title} ::  href: {market.href}')
            long_logger.error(f'MaxRetryError: {exc_type.__name__} ::  market: {market.title} ::  href: {market.href}', exc_info=True)
            continue
        
        except NoSuchWindowException:
            need_reload_driver = True
            exc_type, exc_obj, exc_tb = sys.exc_info()
            short_logger.error(f'MaxRetryError: {exc_type.__name__} ::  market: {market.title} ::  href: {market.href}')
            long_logger.error(f'MaxRetryError: {exc_type.__name__} ::  market: {market.title} ::  href: {market.href}', exc_info=True)
            continue
        
        except WebDriverMemoryOut:
            need_reload_driver = True
            exc_type, exc_obj, exc_tb = sys.exc_info()
            short_logger.warning(f'WebDriverMemoryOut: {exc_type.__name__} ::  market: {market.title} ::  href: {market.href}')
            short_logger.warning(f'WebDriverMemoryOut: {exc_type.__name__} ::  market: {market.title} ::  href: {market.href}', exc_info=True)
            continue
        
        except WebDriverException:
            need_reload_driver = True
            exc_type, exc_obj, exc_tb = sys.exc_info()
            short_logger.critical(f'WebDriverException: {exc_type.__name__} ::  market: {market.title} ::  href: {market.href}')
            long_logger.critical(f'WebDriverException: {exc_type.__name__} ::  market: {market.title} ::  href: {market.href}', exc_info=True)
            continue
        
        # except Exception as ex:
        #     need_reload_driver = True
        #     exc_type, exc_obj, exc_tb = sys.exc_info()
        #     short_logger.critical(f'Exception: {exc_type.__name__} ::  market: {market.title} ::  href: {market.href}')
        #     long_logger.critical(f'Exception: {exc_type.__name__} ::  market: {market.title} ::  href: {market.href}', exc_info=True)
        #     continue
            
        else:
            break
    if need_return_driver:
        return out_list, driver
    else:
        return out_list, None

## 4.2. Выполнение

In [17]:
kill_chrome()
driver = webdriver.Chrome()
driver.maximize_window()

In [18]:
dead_line = 20
out_list = list()

catalog_deque = deque(market_catalog_links[:])

with tqdm(total=len(catalog_deque)) as pbar:
    while len(catalog_deque) > 0:
        if not dead_line:
            break
        
        catalog_link = catalog_deque.popleft()
        try:
            out_data, out_driver = get_data(driver, catalog_link, limitMB=70000)
        except MyLinkDifferError:
            catalog_deque.append(catalog_link)
            continue
        
        if not out_driver is None:
            driver = out_driver
        
        out_list.extend(out_data)

        pbar.update(1)

100%|██████████| 3653/3653 [2:04:06<00:00,  2.04s/it]  


In [20]:
df = pd.concat(out_list, ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44776 entries, 0 to 44775
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        44776 non-null  datetime64[ns]
 1   sity        44776 non-null  object        
 2   market      44776 non-null  object        
 3   SU1         44776 non-null  object        
 4   SU2         44776 non-null  object        
 5   SU3         39351 non-null  object        
 6   title       44776 non-null  object        
 7   date_to     20309 non-null  object        
 8   condition   25542 non-null  object        
 9   price       44776 non-null  float64       
 10  price_last  27728 non-null  float64       
 11  unit        44776 non-null  object        
 12  sale        33957 non-null  object        
 13  vol         24702 non-null  object        
 14  vol_vs      24702 non-null  object        
 15  href        44776 non-null  object        
dtypes: datetime64[ns](1), 

In [86]:
# _save_df_to_zip(df,'result')
mask1 = df['title'].str.lower().str.contains(r'.*колбас.*')
# df[mask1]['vol_vs'].str.extract('(\d+,\d+|\d+)', expand=False).str.replace(',','.').unique()
df[mask1]['vol_vs'].str.extract(r'\ (\W+.+)', expand=False).str.replace(',','.').unique()

array(['₽/кг', nan, '₽/шт'], dtype=object)